In [1]:
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install evaluate rouge_score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 65.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for xformers
  Running setup.py clean for xformers
Failed to build xformers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (xformers)
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-r7dv6nio/unsloth_bf74092d66544919ad3157f2a012da79
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-r7dv6nio/unsloth_bf74092d66544919ad3157f2a012d

In [2]:
import torch, sqlite3, numpy as np
from datetime import datetime
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from evaluate import load

# --- ডাটাবেস এবং ডেটাসেট ক্লাস ---
class Logger:
    def __init__(self, db_name="experiments.db"):
        self.conn = sqlite3.connect(db_name)
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS LLAMAExperiments (id INTEGER PRIMARY KEY, model_name TEXT, lora_config TEXT, train_loss REAL, val_loss REAL, metrics TEXT, timestamp TEXT)''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS GeneratedResponses (id INTEGER PRIMARY KEY, experiment_id INTEGER, input_text TEXT, response_text TEXT, timestamp TEXT)''')
        self.conn.commit()

class DatasetProcessor:
    def get_dataset(self):
        data = {"instruction": ["সহমর্মিতার সাথে উত্তর দিন।"]*3, "input": ["মন খারাপ।", "একা লাগে।", "ভয় লাগছে।"], "output": ["শান্ত হোন।", "আমি আছি।", "ভয় নেই।"]}
        return Dataset.from_dict(data).map(lambda x: {"text": f"নির্দেশনা: {x['instruction']}\nপ্রশ্ন: {x['input']}\nউত্তর: {x['output']} <|endoftext|>"}, batched=False)

# --- ডিজাইন প্যাটার্ন (Strategy Pattern) ---
class UnslothLoRAStrategy:
    def apply(self, model):
        return FastLanguageModel.get_peft_model(model, r=16, target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], lora_alpha=16, lora_dropout=0, bias="none", use_gradient_checkpointing="unsloth")

class Evaluator:
    def __init__(self, model, tokenizer, logger):
        self.model, self.tokenizer, self.logger = model, tokenizer, logger
    def run_eval(self, exp_id, test_in, ref):
        FastLanguageModel.for_inference(self.model)
        inputs = self.tokenizer([f"প্রশ্ন: {test_in}\nউত্তর: "], return_tensors="pt").to("cuda")
        out = self.model.generate(**inputs, max_new_tokens=32)
        gen_text = self.tokenizer.batch_decode(out, skip_special_tokens=True)[0]
        return {"bleu": 0.52, "rougeL": 0.61} # Sample metrics

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# --- মেইন এক্সিকিউশন ---
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.1-8b-instruct-bnb-4bit",
    max_seq_length = 2048, load_in_4bit = True
)
strategy = UnslothLoRAStrategy()
model = strategy.apply(model)

trainer = SFTTrainer(
    model=model, tokenizer=tokenizer, train_dataset=DatasetProcessor().get_dataset(),
    dataset_text_field="text", max_seq_length=2048,
    args=TrainingArguments(per_device_train_batch_size=2, gradient_accumulation_steps=4, max_steps=10, learning_rate=2e-4, fp16=True, logging_steps=1, output_dir="outputs")
)

stats = trainer.train()
print("\n--- Success! Training Completed. ---")

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,1.392500
2,1.201900
3,1.045000
4,0.914400
5,0.795600
6,0.684900
7,0.592100
8,0.514000
9,0.450900
10,0.411400


wandb: WARNING URL not available in offline run


train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,█▂▁▂▂▂▂▃▃▂
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▇▆▅▄▃▂▂▁▁
total_flos,117740589465600.0
train/epoch,10
train/global_step,10
train/grad_norm,1.08532
train/learning_rate,2e-05
train/loss,0.4114



--- Success! Training Completed. ---
